Some functions to help clean up the data.

In [21]:
import torch
import json, os
from tqdm.notebook import tqdm

In [6]:
role_map = {'liberal': 0, 'fascist': 1, 'hitler': 2}
def get_roles(d):
    roles = [d['players'][i]['role'] for i in range(len(d['players']))]
    return list(map(lambda x: role_map[x], roles))

In [7]:
def check_game_end(d):
    roles = get_roles(d)
    libs = 0
    fas = 0
    for turn in d['logs']:
        if 'execution' in turn and roles[turn['execution']] == 2:
            return True
        if 'enactedPolicy' not in turn:
            continue
        if turn['enactedPolicy'] == 'liberal':
            libs += 1
        elif turn['enactedPolicy'] == 'fascist':
            fas += 1
    return libs == 5 or fas == 6


def filter(directory='data/games', out='data/filtered'):
    print('Filtering files...')

    if not os.path.isdir(out):
        os.makedirs(out)

    game_settings = {'deckState': {'lib': 6, 'fas': 11},
                     'trackState': {'lib': 0, 'fas': 0},
                     'powers': [None, 'investigate', 'election', 'bullet', 'bullet'],
                     'enabled': False,
                     'hitlerZone': 3,
                     'vetoZone': 5,
                     'fascistCount': 2,
                     'hitKnowsFas': False}

    i = 0
    for file in os.listdir(directory):
        d = json.load(open(os.path.join(directory, file)))
        if len(d["players"]) == 7 and d['customGameSettings'] == game_settings and check_game_end(d):
            i += 1
            json.dump(d, open(os.path.join(out, f'{i}.json'), 'w'))

In [8]:
def turn_inputs(turn):
    ### Part 1: voting
    inpts = torch.zeros(54)
    i = 10
    inpts[10 + turn["presidentId"]] = 1
    inpts[17 + turn["chancellorId"]] = 1
    for i, v in enumerate(turn["votes"]):
        inpts[24+i] = int(v)*2-1
    yield inpts
    ### Part 2: claims
    if "presidentHand" in turn or 'enactedPolicy' in turn:
        inpts = torch.zeros(54)
        # Set the "claim" neuron to true
        inpts[31] = 1
        if "presidentClaim" in turn:
            claim = turn["presidentClaim"]
            inpts[32:35] = -1
            if claim["blues"] == 0:
                pass
            elif claim["blues"] == 1:
                inpts[32:33] = 1
            elif claim["blues"] == 2:
                inpts[32:34] = 1
            else:
                inpts[32:35] = 1
        
        if "chancellorClaim" in turn:
            claim = turn["chancellorClaim"]
            inpts[35:37] = -1
            if claim["blues"] == 0:
                pass
            elif claim["blues"] == 1:
                inpts[35:36] = 1
            else:
                inpts[35:37] = 1
        
        if 'enactedPolicy' in turn:
            if turn["enactedPolicy"] == "liberal":
                inpts[37] = 1
            else:
                inpts[37] = -1
            
        yield inpts
    ### Part 3+4: investigation
    if "investigationId" in turn:
        inpts = torch.zeros(54)
        inpts[38 + turn["investigationId"]] = 1
        yield inpts
        inpts = torch.zeros(54)
        if "investigationClaim" in turn:
            if turn["investigationClaim"] == "liberal":
                inpts[45] = 1
            else:
                inpts[46] = 1
            print("Invest happened", inpts[45:47])
        yield inpts
    ### Part 5: executions
    if "execution" in turn:
        inpts = torch.zeros(54)
        inpts[47 + turn["execution"]] = 1
        yield inpts
    
    

def make_inputs(d):
    roles = get_roles(d)
    for seat, role in enumerate(roles):
        inputs = []
        ### Part 0: Seat and role
        inpts = torch.zeros(54)
        inpts[seat] = 1
        inpts[7 + role] = 1
        inputs.append(inpts.tolist())
        for turn in d["logs"]:
            for inpts in turn_inputs(turn):
                inputs.append(inpts.tolist())
        yield roles, inputs

In [25]:
def partial_inputs(d):
    roles = get_roles(d)
    inputs = []
    for turn in d["logs"]:
        inpts = [0 for i in range(9)]
        inpts[turn["presidentId"]] = 1
        inpts[turn["chancellorId"]] = 1
        if "enactedPolicy" in turn:
            if turn["enactedPolicy"] == 'liberal':
                inpts[7] = 1
            else:
                inpts[8] = 1
        inputs.append(inpts)
    return roles, inputs

def partial_parse(directory="data/filtered", out="data/partial_data"):
    counts = dict()
    progress_bar = tqdm(os.listdir(directory))
    for file in progress_bar:
        d = json.load(open(os.path.join(directory, file)))
        l = len(d["logs"])
        roles, inputs = partial_inputs(d)
        if l in counts:
            counts[l] += 1
        else:
            counts[l] = 1
        out_dir = os.path.join(out, str(l))
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
        out_path = os.path.join(out_dir, f'{counts[l]}.txt')
        json.dump({'roles': roles, 'inputs': inputs}, open(out_path, "w"))
        

In [23]:
def parse(directory="data/filtered", out="data/parsed"):
    if not os.path.isdir(out):
        os.makedirs(out)
    
    i = -1
    for file in os.listdir(directory):
        d = json.load(open(os.path.join(directory, file)))
        for roles, inputs in make_inputs(d):
            i += 1
            parsed = {'roles': roles, 'inputs': inputs}
            json.dump(parsed, open(os.path.join(out, f'{i}.txt'), "w"))
        break

In [26]:
partial_parse()